# imports and constants

In [270]:
import pandas as pd

from os.path import join as join_pth
import matplotlib.pyplot as plt
import random
import seaborn as sns
# % matplotlib inline
import traintracker


sns.set_style('whitegrid')
import torch
from torch.utils.data.dataloader import DataLoader
import numpy as np
from src import data_loader,models,model_training,utils,data_preprocessing
device='cuda' if torch.cuda.is_available() else 'cpu'


In [302]:
from importlib import reload,import_module


reload(data_loader)
reload(data_preprocessing)
reload(models)
reload(model_training)
reload(utils)

<module 'src.utils' from 'D:\\2022 acheivments\\Projects\\DeepLearning NanoDegree\\Rossmann-Store-Sales\\src\\utils.py'>

array([[1, 3],
       [4, 6],
       [7, 9]])

In [55]:
raw_dataset_path = "../dataset/raw"
prep_dataset_path="../dataset/prep/lstm_model"
nn_model_weights_pth="../model_weights/lstm_model"
nn_model_train_data_pth="../train_data/lstm_model"

# roadmap
- data preparation after analysis run bulk preprocessing functions that have all preprocessing and feature engineering done at the analysis section
- start with the neural network without lstm layers and get the best score after hyperparameters tuning
    - create data loader for the neural network
    - find the best hyperparameters for the first epochs
    - train the model on the best hyperparameters

# Data preparation

## bulk preprocessing

In [144]:
stores_df = pd.read_csv(join_pth(raw_dataset_path, "store.csv"), low_memory=False)
stores_sales_df = pd.read_csv(join_pth(raw_dataset_path, "train.csv"), low_memory=False)

# Sales bulk preprocessing
stores_sales_df_prep=data_preprocessing.store_sales_prep(stores_sales_df=stores_sales_df)

# Store data bulk preprocessing
stores_data_df_prep=data_preprocessing.store_data_prep(store_data_df=stores_df)
# merge and do bulk preprocessing
merge_prep=data_preprocessing.merge_store_sales(sales_data_df=stores_sales_df_prep,store_data_df=stores_data_df_prep)

# drop closed stores data and open column
merge_prep=data_preprocessing.drop_closed_days(merge_prep)




KeyboardInterrupt



In [ ]:
# we have data from jan 2013 to jul 2015
print(merge_prep.Date.min())
print(merge_prep.Date.max())

print(f"months {2*12 + 7}")
print(f"test months = {int(31*0.3)}")

## train test split

In [ ]:
import datetime
boundary=datetime.datetime.strptime("2015-02-01","%Y-%m-%d")
train_data=merge_prep[merge_prep.Date<boundary]
train_data=train_data.sort_values(by='Date')
train_data=data_preprocessing.drop_extra_cols(train_data)
train_data=data_preprocessing.hot_encoding(train_data)

test_data=merge_prep[merge_prep.Date>=boundary]
test_data=test_data.sort_values(by='Date')
test_data=data_preprocessing.drop_extra_cols(test_data)
test_data=data_preprocessing.hot_encoding(test_data)

print(f"train data size {len(train_data)}")
print(f"test data size  {len(test_data)}")

In [ ]:
train_data.info()

In [12]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167269 entries, 76916 to 0
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Store                 167269 non-null  float64
 1   Sales                 167269 non-null  float64
 2   Promo                 167269 non-null  float64
 3   SchoolHoliday         167269 non-null  float64
 4   month                 167269 non-null  float64
 5   day                   167269 non-null  float64
 6   CompetitionDistance   167269 non-null  float64
 7   Promo2                167269 non-null  float64
 8   Promo2Since           167269 non-null  float64
 9   CompetitionOpenSince  167269 non-null  float64
 10  isPromoMonth          167269 non-null  float64
 11  DayOfWeek_1           167269 non-null  float64
 12  DayOfWeek_2           167269 non-null  float64
 13  DayOfWeek_3           167269 non-null  float64
 14  DayOfWeek_4           167269 non-null  float64
 15  D

In [13]:
# no Christmas days in test data so, we will add zero column to the test data
test_data['StateHoliday_christmas']=0.0
test_data=test_data[train_data.columns]
test_data.columns

Index(['Store', 'Sales', 'Promo', 'SchoolHoliday', 'month', 'day',
       'CompetitionDistance', 'Promo2', 'Promo2Since', 'CompetitionOpenSince',
       'isPromoMonth', 'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3',
       'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
       'StateHoliday_christmas', 'StateHoliday_easter', 'StateHoliday_public',
       'StoreType_a', 'StoreType_b', 'StoreType_c', 'StoreType_d',
       'Assortment_a', 'Assortment_b', 'Assortment_c'],
      dtype='object')

## arrange train columns

<img src="./assets/lstm_nn.png"  alt="./assets/lstm_nn.png"/>

In [16]:
print(len(train_data.columns))
print(len(test_data.columns))

28
28


In [15]:
train_data.head()

,Store,Sales,Promo,SchoolHoliday,month,day,CompetitionDistance,Promo2,Promo2Since,CompetitionOpenSince,...,StateHoliday_christmas,StateHoliday_easter,StateHoliday_public,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c
621467,682.0,3375.0,0.0,1.0,1.0,1.0,5.010635,0.0,0.000000,0.000000,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
386137,423.0,9643.0,0.0,1.0,1.0,1.0,7.146772,0.0,0.000000,2.772589,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
700661,769.0,5035.0,0.0,1.0,1.0,1.0,6.733402,1.0,0.693147,0.000000,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
77677,85.0,4220.0,0.0,1.0,1.0,1.0,7.533694,0.0,0.000000,0.000000,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
305081,335.0,2401.0,0.0,1.0,1.0,1.0,4.499810,1.0,0.000000,0.000000,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [17]:
test_data.head()

,Store,Sales,Promo,SchoolHoliday,month,day,CompetitionDistance,Promo2,Promo2Since,CompetitionOpenSince,...,StateHoliday_christmas,StateHoliday_easter,StateHoliday_public,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c
76916,85.0,13899.0,0.0,0.0,2.0,1.0,7.533694,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
863366,948.0,9867.0,0.0,0.0,2.0,1.0,7.265430,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
666908,733.0,18263.0,0.0,0.0,2.0,1.0,6.756932,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
237578,262.0,30525.0,0.0,0.0,2.0,1.0,7.073270,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
699900,769.0,13823.0,0.0,0.0,2.0,1.0,6.733402,1.0,3.295837,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


## save train test data

In [18]:
# train_data
#   sales data
file_name="merged_sales_train.csv"
train_data.to_csv(join_pth(prep_dataset_path,file_name),index=False)


# test_data
#   sales data
file_name="merged_sales_test.csv"
test_data.to_csv(join_pth(prep_dataset_path,file_name),index=False)



# load data

In [250]:
file_name="merged_sales_train.csv"
merged_sales_train=pd.read_csv(join_pth(prep_dataset_path,file_name))


file_name="merged_sales_test.csv"
merged_sales_test=pd.read_csv(join_pth(prep_dataset_path,file_name))


In [58]:
merged_sales_train[(merged_sales_train.year==2014)&(merged_sales_train.Store==622)&(merged_sales_train.month==9)]

,Sales,Promo,SchoolHoliday,month,day,CompetitionDistance,Promo2,Promo2Since,CompetitionOpenSince,isPromoMonth,...,StateHoliday_christmas,StateHoliday_easter,StateHoliday_public,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c
0,13899.0,0.0,0.0,2.0,1.0,7.533694,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,9867.0,0.0,0.0,2.0,1.0,7.265430,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,18263.0,0.0,0.0,2.0,1.0,6.756932,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,30525.0,0.0,0.0,2.0,1.0,7.073270,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,13823.0,0.0,0.0,2.0,1.0,6.733402,1.0,3.295837,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167264,11449.0,1.0,1.0,7.0,31.0,5.192957,1.0,3.367296,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
167265,5826.0,1.0,0.0,7.0,31.0,7.774856,1.0,2.197225,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
167266,7537.0,1.0,1.0,7.0,31.0,6.507278,1.0,1.098612,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
167267,9063.0,1.0,0.0,7.0,31.0,7.635304,1.0,4.060443,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


# Train loader

In [303]:
train_dataset=data_loader.LSTMSalesDataset(merged_sales_train,seq_length=30)
test_dataset=data_loader.LSTMSalesDataset(merged_sales_test,seq_length=30)

In [304]:
# data loader test
iterr=iter(train_dataset)


In [305]:
lstm_in,nn_in,out=next(iterr)
print(f"lstm in shape --> {lstm_in.shape} nn in shape --> {nn_in.shape} out-> shape {out.shape}")

lstm in shape --> torch.Size([30, 15]) nn in shape --> torch.Size([26]) out-> shape torch.Size([1])


In [306]:
lstm_in[-1]

tensor([7.4480e+03, 1.0000e+00, 0.0000e+00, 2.0000e+00, 5.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00])

In [311]:
nn_in

tensor([1.0000, 0.0000, 2.0000, 6.0000, 9.8336, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000])

In [307]:
train_dataset.current_store

343.0

In [308]:
merged_sales_train[merged_sales_train.Store==train_dataset.current_store].iloc[:31]

,Store,Sales,Promo,SchoolHoliday,month,day,CompetitionDistance,Promo2,Promo2Since,CompetitionOpenSince,...,StateHoliday_christmas,StateHoliday_easter,StateHoliday_public,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c
244,343.0,6447.0,0.0,1.0,1.0,2.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1943,343.0,5781.0,0.0,1.0,1.0,3.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3055,343.0,6346.0,0.0,1.0,1.0,4.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4310,343.0,5621.0,0.0,0.0,1.0,5.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4955,343.0,10701.0,1.0,0.0,1.0,7.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5717,343.0,7766.0,1.0,0.0,1.0,8.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
7424,343.0,7129.0,1.0,0.0,1.0,9.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
8182,343.0,7245.0,1.0,0.0,1.0,10.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
9756,343.0,7428.0,1.0,0.0,1.0,11.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
10449,343.0,5905.0,0.0,0.0,1.0,12.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [312]:
out

tensor([8432.])

In [98]:
lstm_in[:,0]

tensor([ 8007.,  6685.,  8196., 10121., 10634.,  8281.,  8789.,  8723.,  8676.,
        10206.,  6984.,  6366.,  5407.,  5877.,  6657., 10446.,  8195.,  9965.,
         8658.,  8139.,  8820., 11006.,  6512.,  6249.,  6505.,  7634.,  8233.,
        12153.,  8598., 10057.])

In [309]:
out

tensor([8432.])

In [310]:
merged_sales_train[merged_sales_train.Store==train_dataset.current_store].iloc[:31]

,Store,Sales,Promo,SchoolHoliday,month,day,CompetitionDistance,Promo2,Promo2Since,CompetitionOpenSince,...,StateHoliday_christmas,StateHoliday_easter,StateHoliday_public,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c
244,343.0,6447.0,0.0,1.0,1.0,2.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1943,343.0,5781.0,0.0,1.0,1.0,3.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3055,343.0,6346.0,0.0,1.0,1.0,4.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4310,343.0,5621.0,0.0,0.0,1.0,5.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4955,343.0,10701.0,1.0,0.0,1.0,7.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
5717,343.0,7766.0,1.0,0.0,1.0,8.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
7424,343.0,7129.0,1.0,0.0,1.0,9.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
8182,343.0,7245.0,1.0,0.0,1.0,10.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
9756,343.0,7428.0,1.0,0.0,1.0,11.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
10449,343.0,5905.0,0.0,0.0,1.0,12.0,9.833601,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [285]:
batch_size=1
train_loader=DataLoader(train_dataset,batch_size=batch_size,drop_last=True)
test_loader=DataLoader(test_dataset,batch_size=batch_size,drop_last=True)

In [286]:
# data loader test
iterr=iter(train_loader)
lstm_in,nn_in,out=next(iterr)

print(f"lstm in shape --> {lstm_in.shape} nn in shape --> {nn_in.shape} out-> shape {out.shape}")



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

# model training

In [209]:
lstm_architecture={"input_size":train_dataset.no_lstm_cols,"num_layers":1,"hidden_size":128}
nn_hidden=[512,256]
nn_architecture={"input_size":train_dataset.no_nn_cols,"hidden_shape":nn_hidden}
lstm_model=models.SalesLstm(lstm_architecture,nn_architecture)
lstm_model.to(device)

SalesLstm(
  (lstm): LSTM(15, 128, batch_first=True)
  (fcn): Sequential(
    (0): Linear(in_features=154, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=1, bias=True)
    (5): ReLU()
  )
)

In [210]:
state_dict=torch.load(traintracker.latest_weights_path(nn_model_weights_pth),map_location='cpu')
lstm_model.load_state_dict(state_dict)


<All keys matched successfully>

In [51]:

out,lstm_hidden=lstm_model(lstm_in,nn_in)

In [52]:
out.shape

torch.Size([32, 1])

In [53]:
train_losses,valid_losses=model_training.lstm_train(lstm_model,train_loader,test_loader,1,last_weights=True,train_data_dir=nn_model_train_data_pth,weights_dir=nn_model_weights_pth,device=device)

Testing before training
 testing [..........] time remaining (m) = 9.5 Avg Test_Loss=329156.8771186414

KeyboardInterrupt: 

In [158]:
stores_df = pd.read_csv(join_pth(raw_dataset_path, "store.csv"), low_memory=False)
kaggle_sales_df = pd.read_csv(join_pth(raw_dataset_path, "test.csv"), low_memory=False)
# kaggle_sales_df.drop('Id',inplace=True,axis=1)
# Sales bulk preprocessing
kaggle_sales_df=data_preprocessing.store_sales_prep(stores_sales_df=kaggle_sales_df)

# Store data bulk preprocessing
stores_data_df_prep=data_preprocessing.store_data_prep(store_data_df=stores_df)
# merge and do bulk preprocessing
merge_prep=data_preprocessing.merge_store_sales(sales_data_df=kaggle_sales_df,store_data_df=stores_data_df_prep)

# drop closed stores data and open column
merge_prep=data_preprocessing.drop_closed_days(merge_prep)

In [159]:
merge_prep.head()

,Id,Store,DayOfWeek,Date,Promo,StateHoliday,SchoolHoliday,year,month,day,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,CompetitionOpenDate,Promo2Since,CompetitionOpenSince,isPromoMonth
20543,40660,553,6,2015-08-01,0,NaN,0,2015,8,1,c,a,8.019613,1,"Jan,Apr,Jul,Oct",2002-09-01,4.174387,0,False
14543,40535,394,6,2015-08-01,0,NaN,1,2015,8,1,d,a,9.291920,0,NaN,2013-01-01,0.000000,0,False
14495,40534,393,6,2015-08-01,0,NaN,0,2015,8,1,d,c,8.480529,0,NaN,2008-03-01,0.000000,0,False
14447,40533,392,6,2015-08-01,0,NaN,0,2015,8,1,a,a,7.659171,0,NaN,2013-01-01,0.000000,0,False
14399,40532,391,6,2015-08-01,0,NaN,0,2015,8,1,a,a,6.131226,1,"Feb,May,Aug,Nov",2014-11-01,3.178054,0,True


In [160]:
merge_prep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35104 entries, 20543 to 0
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Id                    35104 non-null  int64         
 1   Store                 35104 non-null  int64         
 2   DayOfWeek             35104 non-null  category      
 3   Date                  35104 non-null  datetime64[ns]
 4   Promo                 35104 non-null  category      
 5   StateHoliday          26 non-null     object        
 6   SchoolHoliday         35104 non-null  category      
 7   year                  35104 non-null  int64         
 8   month                 35104 non-null  int64         
 9   day                   35104 non-null  int64         
 10  StoreType             35104 non-null  category      
 11  Assortment            35104 non-null  category      
 12  CompetitionDistance   35104 non-null  float64       
 13  Promo2          

In [161]:
train_data=merge_prep
train_data=train_data.sort_values(by='Date')
train_data=train_data.drop(['year', 'Date', 'PromoInterval', 'CompetitionOpenDate'],axis=1)
train_data=data_preprocessing.hot_encoding(train_data)

In [162]:
train_data['Sales']=-1

In [163]:
train_data.head()

,Id,Store,Promo,SchoolHoliday,month,day,CompetitionDistance,Promo2,Promo2Since,CompetitionOpenSince,...,DayOfWeek_7,StateHoliday_public,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,Sales
20543,40660.0,553.0,0.0,0.0,8.0,1.0,8.019613,1.0,4.174387,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1
40415,41074.0,1099.0,0.0,0.0,8.0,1.0,5.298317,1.0,3.332205,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1
40367,41073.0,1097.0,0.0,0.0,8.0,1.0,6.579251,0.0,0.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,-1
40319,41072.0,1096.0,0.0,0.0,8.0,1.0,7.029973,1.0,2.833213,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,-1
40271,41071.0,1094.0,0.0,0.0,8.0,1.0,7.774856,1.0,2.302585,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,-1


In [164]:
train_data2=pd.concat([test_data,train_data])

In [165]:
train_data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 202373 entries, 76916 to 0
Data columns (total 29 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Store                   202373 non-null  float64
 1   Sales                   202373 non-null  float64
 2   Promo                   202373 non-null  float64
 3   SchoolHoliday           202373 non-null  float64
 4   month                   202373 non-null  float64
 5   day                     202373 non-null  float64
 6   CompetitionDistance     202373 non-null  float64
 7   Promo2                  202373 non-null  float64
 8   Promo2Since             202373 non-null  float64
 9   CompetitionOpenSince    202373 non-null  float64
 10  isPromoMonth            202373 non-null  float64
 11  DayOfWeek_1             202373 non-null  float64
 12  DayOfWeek_2             202373 non-null  float64
 13  DayOfWeek_3             202373 non-null  float64
 14  DayOfWeek_4          

In [166]:
train_data2[train_data2.Store==1]

,Store,Sales,Promo,SchoolHoliday,month,day,CompetitionDistance,Promo2,Promo2Since,CompetitionOpenSince,...,StateHoliday_easter,StateHoliday_public,StoreType_a,StoreType_b,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,Id
179,1.0,6038.0,1.0,0.0,2.0,2.0,7.146772,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
178,1.0,4901.0,1.0,0.0,2.0,3.0,7.146772,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
177,1.0,4672.0,1.0,0.0,2.0,4.0,7.146772,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
176,1.0,4394.0,1.0,0.0,2.0,5.0,7.146772,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
175,1.0,5022.0,1.0,0.0,2.0,6.0,7.146772,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,1.0,-1.0,0.0,0.0,9.0,12.0,7.146772,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4281.0
3,1.0,-1.0,1.0,0.0,9.0,14.0,7.146772,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2569.0
2,1.0,-1.0,1.0,0.0,9.0,15.0,7.146772,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1713.0
1,1.0,-1.0,1.0,0.0,9.0,16.0,7.146772,0.0,0.0,0.0,...,NaN,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,857.0


In [234]:

store_idx=2
seq_len=30
# hidden=None
store_sales=train_data2[train_data2.Store==store_idx]
prev_data=store_sales.iloc[0:30]
new_data=store_sales.iloc[30]

hidden=None
iterr=iter(train_loader)
lstm_in,nn_in,target=next(iterr)

pred_out,hidden=lstm_model.predict(prev_data,new_data,hidden,device)

In [235]:
pred_out

tensor([[0.]])

In [232]:
hidden=None
iterr=iter(train_loader)
lstm_in,nn_in,target=next(iterr)
lstm_model.eval()
with torch.no_grad():
    pred_out,hidden=lstm_model(lstm_in,nn_in)

In [233]:
pred_out[0]

tensor([7252.0068])

In [183]:
seq_len=30


for store_idx in train_data2.Store.unique():
    store_sales=train_data2[train_data2.Store==store_idx]
    hidden=None
    no_seq=len(store_sales)-seq_len
    out_idx=seq_len
    for i in range(no_seq):
        prev_data=store_sales.iloc[i:i + seq_len]
        new_data=store_sales.iloc[out_idx]
        out_idx+=1
        pred_out,hidden=predict(prev_data,new_data,hidden,device)

        if new_data['Sales']==-1:
            new_data.loc['Sales']=out.item()
            x=1



KeyboardInterrupt: 

In [185]:
submit_data = pd.read_csv(join_pth(raw_dataset_path, "sample_submission.csv"), low_memory=False)
submit_data.set_index('Id',inplace=True)

In [187]:
submit_data.loc[5]=5

In [188]:
submit_data

,Sales
Id,
1,0
2,0
3,0
4,0
5,5
...,...
41084,0
41085,0
41086,0


In [189]:
store_idx=1
seq_len=30
hidden=None
store_sales=train_data2[train_data2.Store==store_idx]
prev_data=store_sales.iloc[0:30]
new_data=store_sales.iloc[30]


pred_out,hidden=predict(prev_data,new_data,hidden,device)

In [191]:
pred_out

tensor([[0.]])